In [1]:
import pandas as pd
import numpy as np

In [2]:
sampled_pwds = pd.read_csv("tl/e24_random_10_pwmds.csv")

In [3]:
model_name = "resnet"

In [4]:
nn_pareto_df = pd.read_csv(f"/home/saisamarth/exp/tl/e24_{model_name}_NN_pareto_powermodes_pred_250_sampled.csv")
nn_df = pd.read_csv(f"/home/saisamarth/exp/tl/e24_merged_observed_predicted_{model_name}_NN_sampled_250.csv")

In [5]:
sampled_pwds = list(sampled_pwds['powermode'].values)

In [6]:
random_df = nn_df[nn_df['powermode'].isin(sampled_pwds)]


In [9]:
nn_df

,observed_time,observed_power,powermode,predicted_time,predicted_power
0,216.409889,22.8145,4_422400_1300500000_2133000000,305.87840,25.699232
1,257.015106,22.5720,4_422400_1300500000_3199000000,285.08035,31.153877
2,299.540268,22.1700,4_422400_1300500000_665600000,306.90042,23.329690
3,254.655281,20.8105,4_422400_1134750000_2133000000,304.57925,24.074755
4,233.582397,20.4630,4_422400_1134750000_3199000000,292.16675,27.793627
...,...,...,...,...,...
436,171.996704,23.8930,12_2201600_318750000_3199000000,169.34067,24.942556
437,236.020462,20.2730,12_2201600_318750000_665600000,230.96603,18.885178
438,465.527954,17.7560,12_2201600_114750000_2133000000,449.68350,18.024520
439,464.036285,18.5690,12_2201600_114750000_3199000000,434.87677,20.497835


In [73]:
random_df.reset_index(drop=True, inplace=True)

In [74]:
# drop predicted columns
random_df = random_df.drop(columns=['predicted_time', 'predicted_power'])

In [75]:
if model_name == "bert":
    power_list = np.arange(10, 61, 1)
else:
    power_list = np.arange(10, 51, 1)

In [76]:
result_df_random = pd.DataFrame(columns=['powermode', 'observed_time', 'observed_power', 'power_budget'])
for power_budget in power_list:
    # find the best time under power budget for each power_budget
    try:
        best_time = random_df[random_df['observed_power'] <= power_budget].sort_values(by='observed_time').iloc[0]
        result_df_random.loc[len(result_df_random)] = [best_time['powermode'], best_time['observed_time'], best_time['observed_power'], power_budget]
    except:
        result_df_random.loc[len(result_df_random)] = [np.nan, np.nan, np.nan, power_budget]


In [77]:
result_df_nn = pd.DataFrame(columns=['powermode', 'observed_time', 'observed_power', 'power_budget'])
for power_budget in power_list:
    # find the best time under power budget for each power_budget
    try:
        best_time = nn_pareto_df[nn_pareto_df['predicted_power'] <= power_budget].sort_values(by='predicted_time').iloc[0]
        result_df_nn.loc[len(result_df_nn)] = [best_time['powermode'], best_time['observed_time'], best_time['observed_power'], power_budget]
    except:
        result_df_nn.loc[len(result_df_nn)] = [np.nan, np.nan, np.nan, power_budget]


In [78]:
# merge the two dataframes on power_budget with suffixes (random, nn)
result_df = pd.merge(result_df_random, result_df_nn, on='power_budget', suffixes=('_random', '_nn'))

In [79]:
result_df.to_csv(f"{model_name}_random_vs_nn_train_alt10.csv", index=False)